# Zep Cloud
> Recall, understand, and extract data from chat histories. Power personalized AI experiences.

> [Zep](https://www.getzep.com) is a long-term memory service for AI Assistant apps.
> With Zep, you can provide AI assistants with the ability to recall past conversations, no matter how distant,
> while also reducing hallucinations, latency, and cost.

> See [Zep Cloud Installation Guide](https://help.getzep.com/sdks)

## Usage

In the examples below, we're using Zep's auto-embedding feature which automatically embeds documents on the Zep server 
using low-latency embedding models.

## Note
- These examples use Zep's async interfaces. Call sync interfaces by removing the `a` prefix from the method names.

## Load or create a Collection from documents

In [1]:
from uuid import uuid4

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import ZepCloudVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

ZEP_API_KEY = "<your zep project key>"  # You can generate your zep project key from the Zep dashboard
collection_name = f"babbage{uuid4().hex}"  # a unique collection name. alphanum only

# load the document
article_url = "https://www.gutenberg.org/cache/epub/71292/pg71292.txt"
loader = WebBaseLoader(article_url)
documents = loader.load()

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Instantiate the VectorStore. Since the collection does not already exist in Zep,
# it will be created and populated with the documents we pass in.
vs = ZepCloudVectorStore.from_documents(
    docs,
    embedding=None,
    collection_name=collection_name,
    api_key=ZEP_API_KEY,
)

In [2]:
# wait for the collection embedding to complete


async def wait_for_ready(collection_name: str) -> None:
    import time

    from zep_cloud.client import AsyncZep

    client = AsyncZep(api_key=ZEP_API_KEY)

    while True:
        c = await client.document.get_collection(collection_name)
        print(
            "Embedding status: "
            f"{c.document_embedded_count}/{c.document_count} documents embedded"
        )
        time.sleep(1)
        if c.document_embedded_count == c.document_count:
            break


await wait_for_ready(collection_name)

Embedding status: 401/401 documents embedded


## Simarility Search Query over the Collection

In [4]:
# query it
query = "what is the structure of our solar system?"
docs_scores = await vs.asimilarity_search_with_relevance_scores(query, k=3)

# print results
for d, s in docs_scores:
    print(d.page_content, " -> ", s, "\n====\n")

the positions of the two principal planets, (and these the most
necessary for the navigator,) Jupiter and Saturn, require each not less
than one hundred and sixteen tables. Yet it is not only necessary to
predict the position of these bodies, but it is likewise expedient to
tabulate the motions of the four satellites of Jupiter, to predict the
exact times at which they enter his shadow, and at which their shadows
cross his disc, as well as the times at which they are interposed  ->  0.78691166639328 
====

are reduced to a system of wheel-work. We are, nevertheless, not without
hopes of conveying, even to readers unskilled in mathematics, some
satisfactory notions of a general nature on this subject.

_Thirdly_, To explain the actual state of the machinery at the present
time; what progress has been made towards its completion; and what are
the probable causes of those delays in its progress, which must be a
subject of regret to all friends of science. We shall indicate what  ->  0.785

## Search over Collection Re-ranked by MMR

Zep offers native, hardware-accelerated MMR re-ranking of search results.

In [5]:
query = "what is the structure of our solar system?"
docs = await vs.asearch(query, search_type="mmr", k=3)

for d in docs:
    print(d.page_content, "\n====\n")

the positions of the two principal planets, (and these the most
necessary for the navigator,) Jupiter and Saturn, require each not less
than one hundred and sixteen tables. Yet it is not only necessary to
predict the position of these bodies, but it is likewise expedient to
tabulate the motions of the four satellites of Jupiter, to predict the
exact times at which they enter his shadow, and at which their shadows
cross his disc, as well as the times at which they are interposed 
====

are reduced to a system of wheel-work. We are, nevertheless, not without
hopes of conveying, even to readers unskilled in mathematics, some
satisfactory notions of a general nature on this subject.

_Thirdly_, To explain the actual state of the machinery at the present
time; what progress has been made towards its completion; and what are
the probable causes of those delays in its progress, which must be a
subject of regret to all friends of science. We shall indicate what 
====

general commerce. But the

# Filter by Metadata

Use a metadata filter to narrow down results. First, load another book: "Adventures of Sherlock Holmes"

In [3]:
# Let's add more content to the existing Collection
article_url = "https://www.gutenberg.org/files/48320/48320-0.txt"
loader = WebBaseLoader(article_url)
documents = loader.load()

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

await vs.aadd_documents(docs)

await wait_for_ready(collection_name)

Adding documents
generated documents 1290
added documents ['ddec2883-5776-47be-a7bc-23e851e969a9', '19d2a5ce-0a36-4d32-b522-2a54a69970e8', '5e6d8ed2-4527-4515-9557-f178b2682df8', 'ad2b04d9-e1f1-4d90-9920-ab6d95c63dd7', '19871eb4-b624-49d1-a160-a24487298ac6', '19e9e04b-6b3f-4a91-9717-1565a5065642', '0e9a6411-a6a6-4453-a655-2a251519a1fa', '51de5926-dbb1-4d58-b6d4-47c8e5988042', '244dc860-2bf0-41a1-aa3c-e170c0209c5a', '175e8811-70cd-499c-a35f-2502749bff2e', '5298951a-1087-45f4-afff-3466032301fc', 'd87f4495-95c9-4152-bf1b-90f73652e639', '638f6b85-1e05-462f-8c75-4c642a309d88', '92f7f430-9ffc-4ca5-ac92-953793a4a7b0', '5ce532f2-86b0-4953-8815-96b7fb0da046', '8187857a-02b6-4faa-80b3-ca6d64b0ae59', '599d479c-7c2e-4ad9-b5e4-651fced1f38c', 'e0869e51-92a5-4c87-ba35-a977c3fe43d2', 'bed376d9-3785-4fa3-a986-4d069e817a0e', '31e8e7b1-164b-4872-8a21-0a417d9c2793', '87f96ecb-c78a-4aaf-a976-557a76adecf1', '8f0e7eb2-fb67-4010-bec4-70f1ea319d22', '5d280424-00cc-475c-8334-3ac1dbda84c9', 'a27876c9-68c2-460c-b

We see results from both books. Note the `source` metadata

In [4]:
query = "Was he interested in astronomy?"
docs = await vs.asearch(query, search_type="similarity", k=3)

for d in docs:
    print(d.page_content, " -> ", d.metadata, "\n====\n")

of astronomy, and its kindred sciences, with the various arts dependent
on them. In none are computations more operose than those which
astronomy in particular requires;--in none are preparatory facilities
more needful;--in none is error more detrimental. The practical
astronomer is interrupted in his pursuit, and diverted from his task of
observation by the irksome labours of computation, or his diligence in
observing becomes ineffectual for want of yet greater industry of  ->  {'source': 'https://www.gutenberg.org/cache/epub/71292/pg71292.txt'} 
====

possess all knowledge which is likely to be useful to him in his work,
and this I have endeavored in my case to do. If I remember rightly, you
on one occasion, in the early days of our friendship, defined my limits
in a very precise fashion.”

“Yes,” I answered, laughing. “It was a singular document. Philosophy,
astronomy, and politics were marked at zero, I remember. Botany
variable, geology profound as regards the mud-stains from any 

Now, we set up a filter

In [5]:
filter = {
    "where": {
        "jsonpath": (
            "$[*] ? (@.source == 'https://www.gutenberg.org/files/48320/48320-0.txt')"
        )
    },
}

docs = await vs.asearch(query, search_type="similarity", metadata=filter, k=3)

for d in docs:
    print(d.page_content, " -> ", d.metadata, "\n====\n")

possess all knowledge which is likely to be useful to him in his work,
and this I have endeavored in my case to do. If I remember rightly, you
on one occasion, in the early days of our friendship, defined my limits
in a very precise fashion.”

“Yes,” I answered, laughing. “It was a singular document. Philosophy,
astronomy, and politics were marked at zero, I remember. Botany
variable, geology profound as regards the mud-stains from any region  ->  {'source': 'https://www.gutenberg.org/files/48320/48320-0.txt'} 
====

the evening than in the daylight, for he said that he hated to be
conspicuous. Very retiring and gentlemanly he was. Even his voice was
gentle. He’d had the quinsy and swollen glands when he was young, he
told me, and it had left him with a weak throat, and a hesitating,
whispering fashion of speech. He was always well dressed, very neat and
plain, but his eyes were weak, just as mine are, and he wore tinted
glasses against the glare.”  ->  {'source': 'https://www.gutenber